In [32]:
import pickle

import numpy as np
import pandas as pd
from scipy.stats import spearmanr, kendalltau
from sklearn.linear_model import LogisticRegression, LinearRegression
from tqdm.auto import tqdm, trange

# 1. Data Preparation

In [2]:
with open("players.pkl", "rb") as f:
    players = pd.DataFrame(pickle.load(f)).T.set_index("id").sort_index()
players

,name,patronymic,surname
id,,,
1,Алексей,None,Абабилов
10,Игорь,,Абалов
11,Наталья,Юрьевна,Абалымова
12,Артур,Евгеньевич,Абальян
13,Эрик,Евгеньевич,Абальян
...,...,...,...
224700,Артём,Евгеньевич,Садов
224701,Даниил,Олегович,Трефилов
224702,Владимир,Араратович,Басенцян


In [3]:
with open("results.pkl", "rb") as f:
    results = pickle.load(f)

def process_tournament(tournament, t_id):
    if len(tournament) == 0 or ('mask' not in tournament[0]) or tournament[0]['mask'] is None:
        return None
    out = pd.DataFrame(tournament)
    out['id'] = t_id
    out['team_id'] = out['team'].apply(pd.Series)['id']
    out = out.explode("teamMembers")
    out = out.dropna(subset=['teamMembers'])
    out['player_id'] = out['teamMembers'].apply(lambda x: x['player']['id'])
    return out[['id', 'team_id', 'player_id', 'mask', 'questionsTotal', 'position']]

results = pd.concat([process_tournament(value, key) for key, value in results.items()]).reset_index(drop=True)
results['mask'] = results['mask'].str.replace(r'[^01]', '', regex=True)
results = results.dropna(subset=['mask'])
results

,id,team_id,player_id,mask,questionsTotal,position
0,22,1,1560,0111011101101110001101110011111111110011111100...,67,1.0
1,22,1,2935,0111011101101110001101110011111111110011111100...,67,1.0
2,22,1,3270,0111011101101110001101110011111111110011111100...,67,1.0
3,22,1,4878,0111011101101110001101110011111111110011111100...,67,1.0
4,22,1,18935,0111011101101110001101110011111111110011111100...,67,1.0
...,...,...,...,...,...,...
2331713,6456,69918,129706,100101101100100100010110100100010001010,16,6.0
2331714,6456,69918,192901,100101101100100100010110100100010001010,16,6.0
2331715,6456,63129,165962,101000100110000110001000101000010100001,13,7.0
2331716,6456,63129,154624,101000100110000110001000101000010100001,13,7.0


In [4]:
with open("tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)

tournaments = pd.DataFrame(tournaments).T.set_index("id")
tournaments['type'] = tournaments['type'].apply(lambda x: x['name'])
tournaments = tournaments[['name', 'dateStart', 'dateEnd', 'type']]
tournaments = tournaments.loc[tournaments['type'].isin(["Обычный", "Синхрон", "Строго синхронный"])]
tournaments = tournaments.dropna()
tournaments['dateStart'] = pd.to_datetime(tournaments['dateStart'], utc=True, errors='raise')
tournaments['dateEnd'] = pd.to_datetime(tournaments['dateEnd'], utc=True, errors='raise')
tournaments

,name,dateStart,dateEnd,type
id,,,,
1,Чемпионат Южного Кавказа,2003-07-24 20:00:00+00:00,2003-07-26 20:00:00+00:00,Обычный
2,Летние зори,2003-08-08 20:00:00+00:00,2003-08-08 20:00:00+00:00,Обычный
3,Турнир в Ижевске,2003-11-21 21:00:00+00:00,2003-11-23 21:00:00+00:00,Обычный
4,Чемпионат Украины. Переходной этап,2003-10-10 20:00:00+00:00,2003-10-11 20:00:00+00:00,Обычный
5,Бостонское чаепитие,2003-10-09 20:00:00+00:00,2003-10-12 20:00:00+00:00,Обычный
...,...,...,...,...
6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05 12:00:00+00:00,2020-05-05 15:00:00+00:00,Обычный
6482,Онлайн: 19:00 Зелёный шум,2020-05-07 16:00:00+00:00,2020-05-07 18:30:00+00:00,Обычный
6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08 16:00:00+00:00,2020-05-08 18:30:00+00:00,Обычный


In [5]:
train_tours = tournaments.loc[pd.to_datetime(tournaments['dateStart']).dt.year==2019]
test_tours = tournaments.loc[pd.to_datetime(tournaments['dateStart']).dt.year==2020]
print(f"Train size: {len(train_tours)}, test size: {len(test_tours)}")

Train size: 622, test size: 362


In [6]:
train_res = results.loc[results['id'].isin(train_tours.index)].reset_index(drop=True)
test_res = results.loc[results['id'].isin(test_tours.index)].reset_index(drop=True)

In [7]:
important_players = players.loc[players.index.isin(train_res['player_id']) & players.index.isin(test_res['player_id'])]
players_count = len(important_players)
print(f"{players_count} players played both in 2019 and 2020")

17216 players played both in 2019 and 2020


In [8]:
imp_train_res = train_res.loc[train_res['player_id'].isin(important_players.index)]
imp_test_res = test_res.loc[test_res['player_id'].isin(important_players.index)]

# 2. Baseline

In [9]:
total_questions = 0
for t_id, group in imp_train_res.groupby("id"):
    total_questions += group['mask'].str.len().max()
questions_count = int(total_questions)
print(f"{questions_count} questions in train")

29002 questions in train


In [10]:
acc = []
for t_id, group in tqdm(imp_train_res.groupby("id")):
    curr = pd.DataFrame(group['mask'].apply(list).to_list()).add_prefix("q_")
    curr['player_id'] = group['player_id'].reset_index(drop=True)
    curr = pd.wide_to_long(curr, "q_", i="player_id", j="question_id").reset_index()
    curr['question_id'] = str(t_id) + '_' + curr['question_id'].astype(str)
    curr = curr.rename(columns={"q_": "result"})
    acc.append(curr.dropna())

long_train = pd.concat(acc)
sparse_train_df = pd.get_dummies(long_train, columns=['player_id', 'question_id'], sparse=True)

  0%|          | 0/610 [00:00<?, ?it/s]

In [11]:
assert players_count + questions_count + 1 == len(sparse_train_df.columns)

In [12]:
X_train = sparse_train_df.drop(columns="result").sparse.to_coo().tocsr()
y_train = sparse_train_df['result']

model = LogisticRegression(solver='sag')
model.fit(X_train, y_train)

LogisticRegression(solver='sag')

In [38]:
def get_force_difficulty(train_df, logreg_model):
    player_force = {}
    question_difficulty = {}
    for col, coef in zip(train_df.drop(columns="result").columns, logreg_model.coef_.flatten()):
        if col.startswith("player_id"):
            player_force[int(col[len("player_id_"):])] = coef
        elif col.startswith("question_id"):
            question_difficulty[col[len("question_id_"):]] = -coef
    return player_force, question_difficulty

In [39]:
player_force, question_difficulty = get_force_difficulty(sparse_train_df, model)

In [40]:
important_players['baseline_force'] = important_players.index.map(player_force)
important_players.sort_values('baseline_force', ascending=False).reset_index()

/tmp/ipykernel_47189/131241550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_players['baseline_force'] = important_players.index.map(player_force)


,id,name,patronymic,surname,baseline_force
0,27403,Максим,Михайлович,Руссо,3.379636
1,4270,Александра,Владимировна,Брутер,3.253530
2,28751,Иван,Николаевич,Семушин,3.199325
3,27822,Михаил,Владимирович,Савченков,3.107415
4,30270,Сергей,Леонидович,Спешков,3.048867
...,...,...,...,...,...
17211,211016,Баир,Арсаланович,Батожаргалов,-3.600054
17212,211022,Антон,Игоревич,Козик,-3.600059
17213,211015,Иван,Владимирович,Батурин,-3.600061
17214,211021,Даниил,Александрович,Керченский,-3.600067


Топ игроков выглядит весьма правдоподобно!

# 3. Ranking Evaluation

Для первого приближения силу команды будем считать как среднее сил игроков.
У такого подхода есть очевидный минус в том, что количество игроков в команде не учитывается, и сила одного Семушина будет примерно равна силе полного Корабела, но зато это позволяет спокойно забить на игроков, которые не играли в 2019, и для которых у нас нет рейтинга.

In [24]:
def rank_eval(test_res, player_force):
    eval = test_res.copy()
    eval['player_force'] = eval["player_id"].map(player_force)
    eval = eval.groupby(["id", "team_id", "position"])['player_force'].mean().reset_index().rename(columns={"player_force": "team_force"})
    eval['baseline_rank'] = eval.groupby("id")['team_force'].rank(ascending=False)

    spearman = []
    kendall = []
    for t_id, group in eval.groupby("id"):
        spearman.append(spearmanr(group['position'].values, group['baseline_rank'].values, nan_policy='raise').correlation)
        kendall.append(kendalltau(group['position'].values, group['baseline_rank'].values, nan_policy='raise').correlation)

    return (np.mean(spearman), np.std(spearman)), (np.mean(kendall), np.std(kendall))

In [25]:
spearman, kendall = rank_eval(imp_test_res, player_force)
print(f"Корреляция Спирмена: {spearman[0]:.2f}±{spearman[1]:.2f}")
print(f"Корреляция Кендалла: {kendall[0]:.2f}±{kendall[1]:.2f}")

Корреляция Спирмена: 0.78±0.13
Корреляция Кендалла: 0.62±0.12


# 4. EM-algorithm

Введем скрытую переменную z, которая будет означать, что игрок взял вопрос, играя в конкретной команде. Тогда мы получим поставку precense-only с 0, т.к. если команда не взяла вопрос, то и ни один игрок в ней не взял, а если взяла, то для каждого игрока неизвестно насколько он в этом поучаствовал.

Тогда:
$$ p(z_i=0 | y_i=0, p_i, q_i, \theta) = 1 .$$

Раскроем условную вероятность $z_i=1, y_i=1$:
$$ p(z_i=1, y_i=1 | p_i, q_i, \theta) = p(z_i=1 | y_i=1, p_i, q_i, \theta) * p(y_i=1 | p_i, q_i, \theta). $$

Последний член означает вероятность того, что команда игрока взяла вопрос, её можно выразить так:
$$ p(y_i=1 | p_i, q_i, \theta) = 1 - \prod_{j \in T_i} \left( 1 - p \left(z_j=1, y_j=1 | p_j, q_j, \theta \right) \right), $$ где $j \in T_j$ означает, что произведение идет по игрокам команды игрока $p_i$.

Тогда:
$$ p(z_i=1 | y_i=1, p_i, q_i, \theta) = \frac{ p(z_i=1, y_i=1 | p_i, q_i, \theta) } { 1 - \prod_{j \in T_i} \left( 1 - p \left(z_j=1, y_j=1 | p_j, q_j, \theta \right) \right) }. $$

Таким образом, на M-шаге мы получаем вероятности $p(y_i=1)$ логистической регрессией аналогично бейзлайну, но с таргетом подмененным на $z_i$, и эти вероятности используем на Е-шаге подставляя в формулу выше вместо $ p(z_i=1, y_i=1 | p_i, q_i, \theta) $ (соответствует получению матожидание интегрированием по $z_i$). Для инициализации алгоритма берем вероятности $p(y_i=1)$ из бейзлайна.

In [18]:
acc = []
for t_id, group in tqdm(imp_train_res.groupby("id")):
    curr = pd.DataFrame(group['mask'].apply(list).to_list()).add_prefix("q_")
    curr['player_id'] = group['player_id'].reset_index(drop=True)
    curr['team_id'] = group['team_id'].reset_index(drop=True)
    curr = pd.wide_to_long(curr, "q_", i="player_id", j="question_id").reset_index()
    curr['question_id'] = str(t_id) + '_' + curr['question_id'].astype(str)
    curr = curr.rename(columns={"q_": "result"})
    acc.append(curr.dropna())

long_train_em = pd.concat(acc)

  0%|          | 0/610 [00:00<?, ?it/s]

In [44]:
def sigma(x):
    return 1 / (1 + np.exp(-x))

def inv_sigma(x, eps=1e-8):
    x = np.clip(x, eps, 1-eps)
    return np.log(x / (1-x))

In [45]:
N_ITER = 5

probs = model.predict_proba(X_train)[:, 1]
for i_iter in range(N_ITER):
    # E-step
    long_train_em['p'] = probs
    long_train_em['1-p'] = 1 - probs
    z = long_train_em['p'] / (1 - long_train_em.groupby(['team_id', 'question_id'])['1-p'].transform('prod')) * long_train_em['result'].astype(int)

    # M-step
    # LogisticRegression from sklearn only deals with target-labels, not probas, so we have to use a workaround with LinearRegression
    m_model = LinearRegression().fit(X_train, inv_sigma(z))
    probs = sigma(m_model.predict(X_train))

    player_force, _ = get_force_difficulty(sparse_train_df, m_model)
    spearman, kendall = rank_eval(imp_test_res, player_force)
    print("="*5, f"Итерация {i_iter}", "="*5)
    print(f"Корреляция Спирмена: {spearman[0]:.4f}±{spearman[1]:.4f}")
    print(f"Корреляция Кендалла: {kendall[0]:.4f}±{kendall[1]:.4f}")

===== Итерация 0 =====
Корреляция Спирмена: 0.7727±0.1330
Корреляция Кендалла: 0.6180±0.1240
===== Итерация 1 =====
Корреляция Спирмена: 0.7719±0.1278
Корреляция Кендалла: 0.6160±0.1203
===== Итерация 2 =====
Корреляция Спирмена: 0.7744±0.1220
Корреляция Кендалла: 0.6173±0.1177
===== Итерация 3 =====
Корреляция Спирмена: 0.7744±0.1218
Корреляция Кендалла: 0.6176±0.1174
===== Итерация 4 =====
Корреляция Спирмена: 0.7741±0.1223
Корреляция Кендалла: 0.6170±0.1177


Рост метрик если и есть, то крайне незначительный.

# 5. Tournaments difficulty

## a. Baseline

In [54]:
_, question_difficulty_baseline = get_force_difficulty(sparse_train_df, model)
questions_baseline = pd.DataFrame.from_dict(question_difficulty_baseline, orient='index', columns=['difficulty']).reset_index()
questions_baseline['t_id'] = questions_baseline['index'].str.split('_').str[0].astype(int)
questions_baseline['t_name'] = questions_baseline['t_id'].map(tournaments['name'])
tours_baseline = questions_baseline.groupby(['t_id', 't_name'])['difficulty'].mean().reset_index()
tours_baseline.sort_values("difficulty", ascending=False).head(20)

,t_id,t_name,difficulty
602,6149,Чемпионат Санкт-Петербурга. Первая лига,4.319260
496,5928,Угрюмый Ёрш,2.222249
348,5684,Синхрон высшей лиги Москвы,2.036558
37,5159,Первенство правого полушария,1.981073
580,6101,Воображаемый музей,1.908550
267,5587,Записки охотника,1.681323
7,5025,Кубок городов,1.648818
356,5693,Знание – Сила VI,1.630527
20,5083,Ускользающая сова,1.622329
167,5465,Чемпионат России,1.500946


#### Топ-3 сложнейших

In [49]:
questions_baseline.sort_values("difficulty", ascending=False).head(3)

,index,difficulty,t_id,t_name
7822,5465_39,5.505694,5465,Чемпионат России
7853,5465_67,5.505693,5465,Чемпионат России
1547,5159_34,5.447280,5159,Первенство правого полушария


Топ-1. [Чемпионат России - 2019. Тур 3. Вопрос 40.](https://rc19-quest.livejournal.com/1420.html?thread=17804#t17804)
Топ-2. [Чемпионат России - 2019. Тур 5. Вопрос 68.](https://rc19-quest.livejournal.com/1943.html?thread=25495#t25495)
Топ-3. [Первенство правого полушария, часть XVII. Вопрос 35.](https://db.chgk.info/question/ppp17_u.3/35)

#### Топ-3 простейших

In [50]:
questions_baseline.sort_values("difficulty").head(3)

,index,difficulty,t_id,t_name
1301,5130_18,-5.383069,5130,Лига Сибири. VI тур.
20241,5822_15,-5.341739,5822,ОВСЧ. 5 этап
4884,5402_18,-5.307486,5402,Триптих. Осень


Топ-1. [Студенческая Лига Сибири – 2018/19. VI тур. Тур 2. Вопрос 19.](https://db.chgk.info/question/sls2018vi_u.2/19)
Топ-2. [Открытый Всероссийский синхронный чемпионат — 2019/20. Этап 5. Тур 2. Вопрос 16](https://db.chgk.info/question/ovsch19.5_u.2/16)
Топ-3. [Синхронный турнир «Триптих. Осень». Тур 2. Вопрос 19](https://db.chgk.info/question/triptosen19_u.2/19)

## b. EM

In [53]:
_, question_difficulty_em = get_force_difficulty(sparse_train_df, m_model)
questions_em = pd.DataFrame.from_dict(question_difficulty_em, orient='index', columns=['difficulty']).reset_index()
questions_em['t_id'] = questions_em['index'].str.split('_').str[0].astype(int)
questions_em['t_name'] = questions_em['t_id'].map(tournaments['name'])
tours_em = questions_em.groupby(['t_id', 't_name'])['difficulty'].mean().reset_index()
tours_em.sort_values("difficulty", ascending=False).head(20)

,t_id,t_name,difficulty
602,6149,Чемпионат Санкт-Петербурга. Первая лига,9.475794
496,5928,Угрюмый Ёрш,6.656056
348,5684,Синхрон высшей лиги Москвы,5.840489
37,5159,Первенство правого полушария,5.644635
580,6101,Воображаемый музей,5.641221
504,5942,Чемпионат Мира. Этап 2. Группа В,5.308680
356,5693,Знание – Сила VI,5.289251
167,5465,Чемпионат России,5.257686
20,5083,Ускользающая сова,5.208491
503,5941,Чемпионат Мира. Этап 2. Группа А,5.202664


#### Топ-3 сложнейших

In [55]:
questions_em.sort_values("difficulty", ascending=False).head(3)

,index,difficulty,t_id,t_name
23943,5948_14,13.895656,5948,Чемпионат Мира. Финал. Группа А
23853,5945_14,13.739748,5945,Чемпионат Мира. Этап 3. Группа А
23726,5941_13,13.712981,5941,Чемпионат Мира. Этап 2. Группа А


Топ-1. [XVII чемпионат мира. Тур 7. Вопрос 105](https://db.chgk.info/question/wc19_u.7/105)
Топ-2. [XVII чемпионат мира. Тур 5. Вопрос 75](https://db.chgk.info/question/wc19_u.5/75)
Топ-3. [XVII чемпионат мира. Тур 3. Вопрос 44](https://db.chgk.info/question/wc19_u.3/44)

#### Топ-3 простейших

In [56]:
questions_em.sort_values("difficulty").head(3)

,index,difficulty,t_id,t_name
13918,5614_22,-30.720552,5614,Межфакультетский кубок МГУ. Отбор №6
12596,5578_24,-29.672814,5578,Межфакультетский кубок МГУ. Отбор №1
13205,5593_9,-29.578290,5593,Межфакультетский кубок МГУ. Отбор №3


Отборы межфака проводятся на пакетах из свеченых вопросов, понадерганных из базы, так что искать их особого смысла нет.

# 6. Players

In [62]:
player_force_em, _ = get_force_difficulty(sparse_train_df, m_model)
important_players['em_force'] = important_players.index.map(player_force_em)
important_players['questions_played'] = important_players.index.map(long_train.groupby('player_id')['result'].count().squeeze())

/tmp/ipykernel_47189/3715618250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_players['em_force'] = important_players.index.map(player_force_em)
/tmp/ipykernel_47189/3715618250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_players['questions_played'] = important_players.index.map(long_train.groupby('player_id')['result'].count().squeeze())


In [63]:
important_players.sort_values("baseline_force", ascending=False).head(20)

,name,patronymic,surname,baseline_force,em_force,questions_played
id,,,,,,
27403,Максим,Михайлович,Руссо,3.379636,11.114611,2103
4270,Александра,Владимировна,Брутер,3.253530,10.218169,2581
28751,Иван,Николаевич,Семушин,3.199325,10.113092,3663
27822,Михаил,Владимирович,Савченков,3.107415,9.964644,3104
30270,Сергей,Леонидович,Спешков,3.048867,9.613678,3554
30152,Артём,Сергеевич,Сорожкин,3.020755,10.106900,4777
20691,Станислав,Григорьевич,Мереминский,2.902483,10.134549,1584
18036,Михаил,Ильич,Левандовский,2.848447,9.220273,1457
20207,Михаил,Леонидович,Матвеев,2.822794,9.297500,881


In [64]:
important_players.sort_values("em_force", ascending=False).head(20)

,name,patronymic,surname,baseline_force,em_force,questions_played
id,,,,,,
38175,Максим,Игоревич,Пилипенко,2.490770,17.850452,36
14312,Виталий,Витальевич,Кичкирук,1.300795,16.283550,90
22474,Илья,Сергеевич,Немец,2.253483,16.251323,75
101843,Михаил,Борисович,Волхонский,1.274012,15.340509,35
14996,Ольга,Александровна,Козлова,1.818055,14.416165,36
199775,Денис,Евгеньевич,Кащеев,1.640680,14.347296,36
65825,Олег,Евгеньевич,Шапошников,0.890123,13.521538,309
189502,Никита,Алексеевич,Коновалов,1.743571,12.565181,741
113584,Антон,Владимирович,Козак,0.142638,11.666339,36
